# **Agent as a Tool (in OpenAI Agents SDK)**
[Colab notebook link][1]

[1]: https://colab.research.google.com/drive/1wV4IxJlJIdrmICWVnIrzb8FPhCetPljc?usp=sharing
---

## 1. **First, What’s a Tool?**

* In the **Agents SDK**, a **tool** is a function, API, or skill that an **agent** can call.
* Example: a **calculator tool**, a **weather API tool**, or a **Python code runner tool**.

👉 Normally, **you (the developer)** define tools.

<br>

---

<br>

## 2. **But What if an Agent Itself Becomes a Tool?**

* Sometimes, **an agent can act as a tool** for another agent.
* This means:

  * Agent A (the main agent) can **call Agent B** (which behaves like a specialized tool).
  * Agent B handles the request (using its own tools, logic, knowledge).
  * Agent B returns the result → Agent A gives final response to the user.

👉 This is called **Agent as a Tool**.

<br>

---

<br>

## 3. **Why Do We Need “Agent as a Tool”?**

* To **break down big systems into smaller expert agents**.
* Example:

  * **Main Agent** = General AI assistant.
  * **Finance Agent** = Specialized in handling financial queries.
  * **HR Agent** = Specialized in company policies.
* Instead of giving all tools to one mega-agent, you give each sub-agent specific tools → then plug them into the main agent.

👉 This makes the system **modular, reusable, and easier to manage**.

<br>

---

<br>

## 4. **Workflow Diagram**

```
User → Main Agent → [Finance Agent, HR Agent, Research Agent] → Tools → Answer
```

* Main Agent decides: “This is a finance question → I’ll use Finance Agent.”
* Finance Agent (working as a tool) → fetches answer.
* Main Agent → replies to user.

<br>

---

<br>

## 6. **Benefits of Agent as Tool**

* ✅ **Modularity** → Split complex systems into smaller expert agents.
* ✅ **Reusability** → Same sub-agent can be used across multiple projects.
* ✅ **Scalability** → Easy to add/remove specialized agents.
* ✅ **Organization** → Keeps code and tools clean.

<br>

---

<br>

## 7. **Exam Key Points (Cheat Sheet)**

* **Definition**: Agent as Tool = wrapping an agent so it can be used as a tool by another agent.
* **Purpose**: To build modular, multi-agent systems.
* **How**: Use `.as_tool()` method in the Agents SDK.
* **Workflow**: Main Agent → Sub-Agent (as tool) → Result → User.
* **Benefits**: Modularity, reuse, scalability, clean design.

<br>

---

<br>

## **Summary**:
In the OpenAI Agents SDK, an **Agent can act as a Tool** for another Agent.
This allows building **multi-agent systems** where each agent specializes in a domain, and the main agent coordinates them like a **manager with expert assistants**.

In [4]:
!pip install -Uq openai-agents

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from agents import Agent, AsyncOpenAI, Runner, OpenAIChatCompletionsModel, set_tracing_disabled
from google.colab import userdata

set_tracing_disabled(True)

import asyncio

GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

if not GEMINI_API_KEY:
  raise ValueError("GEMINI_API_KEY is not set")

external_client = AsyncOpenAI(
    api_key= GEMINI_API_KEY,
    base_url= "https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model = "gemini-2.5-flash",
    openai_client = external_client,
)

# ---- Agents as Tool ----
spanish_agent = Agent(
    name= "spanish_agent",
    instructions= "You translate the user's message to spanish.",
    model= model,
)

french_agent = Agent(
    name= "french_agent",
    instructions= "You translate the user's message to french.",
    model= model,
)

urdu_agent = Agent(
    name= "urdu_agent",
    instructions= "You translate the user's message to urdu.",
    model= model,
)

orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions= """
    You are a translation agent. You use the tools given to you to translate.
    If asked for multiple translations, you call the relevant tools in order.
    You never translate on your own, you always use the provided tools.
    """,
    tools= [
        spanish_agent.as_tool(
            tool_name = "spanish_agent",
            tool_description = "Translate the user's message to Spanish."
        ),
        french_agent.as_tool(
            tool_name = "french_agent",
            tool_description = "Translate the user's message to French."
        ),
        urdu_agent.as_tool(
            tool_name = "urdu_agent",
            tool_description = "Translate the user's message to urdu."
        ),
    ],
    model=model
)

async def main():
  msg = input("Hi! What would you like translated, and to which languages?")

  result = await Runner.run(starting_agent=orchestrator_agent, input=msg)
  print(result.final_output)

if __name__ == "__main__":
  asyncio.run(main())

Hi! What would you like translated, and to which languages?"hello my name is ayesha" translate this in french
Bonjour, je m'appelle Ayesha.


# **Handoffs <font color=green> VS </font> Agent as a Tool**

---

## 1. **Handoffs (Definition)**

* A **handoff** means one **agent stops** and **passes full control** of the conversation to another agent.
* The new agent becomes the **primary owner** of the interaction.
* The first agent no longer participates.

👉 Think of it like **transferring a customer call** from one department to another.

---

## 2. **Agent as a Tool (Definition)**

* Here, an **agent is wrapped as a tool**.
* The **main agent stays in control** and can call the sub-agent **whenever needed**.
* The sub-agent behaves like a **specialist function** and returns the result.

👉 Think of it like a **manager consulting a specialist**, then continuing the conversation.

---

## 3. **Workflow Difference**

### Handoff:

```
User ➡ Agent A  ➡ (Handoff) ➡ Agent B ➡  User
                       ⬇
               (Agent A leaves)
```

### Agent as Tool:

```
User ➡ Main Agent ➡ (Calls Sub-Agent as Tool) ➡ Returns to Main Agent ➡ User
            ⬇
       (Main Agent stays in control)
```

---

## 4. **Best Use Cases**

### ✅ Handoffs

* When one agent should **completely give up control** to another.
* Use cases:

  * Customer support routing (HR bot hands off to IT bot).
  * Escalation (Chatbot hands off to a human agent).
  * Multi-department workflows (Banking bot → Loan specialist bot).

👉 Best when: The conversation is **long-term** and the first agent isn’t needed anymore.

---

### **Agent as a Tool**

* When the main agent needs **occasional help** from another expert agent.
* Use cases:

  * General Assistant using a Math Agent for calculations.
  * Research Agent using a Finance Agent to analyze stock reports.
  * Main Agent orchestrating multiple small expert sub-agents.

👉 Best when: You need a **modular system** where the main agent **stays in charge** and just calls helpers.

---

## 5. **Quick Analogy**

* **Handoff** = You call a company → Customer Service says “Let me transfer you to Billing” → now you only talk to Billing.
* **Agent as Tool** = You call a company → Customer Service asks Billing for your invoice → then Customer Service explains it to you.

---

## 6. **Cheat-Sheet (Comparison Table)**

| Feature          | Handoffs 🤝                       | Agent as Tool 🛠                            |
| ---------------- | --------------------------------- | ------------------------------------------- |
| Control          | Original agent **loses control**  | Main agent **retains control**              |
| Interaction Flow | One agent → another takes over    | Main agent → calls sub-agent → continues    |
| Best Use Case    | Customer routing, escalation      | Modular expert helpers, multi-agent systems |
| Analogy          | Call transferred to another dept. | Manager consulting an expert                |

---

✅ **Summary**:

* **Handoffs** = one agent **fully hands over** the conversation.
* **Agent as Tool** = main agent **delegates specific tasks** to a sub-agent but stays in charge.


# **Triage Agent vs Orchestrator Agent**

---

## 1. **Triage Agent (Definition)**

* A **Triage Agent** is like a **gatekeeper**.
* Its job: **decide which specialized agent should handle the user’s request**.
* After choosing, it can either:

  * **Handoff** (pass full control to that agent), or
  * **Route temporarily** (ask an expert agent for info, then return).

👉 Think of it like a **hospital triage nurse**: decides if you go to a doctor, dentist, or cardiologist.

<br>

---

<br>

## 2. **Orchestrator Agent (Definition)**

* An **Orchestrator Agent** is like a **project manager**.
* It **plans and coordinates multi-step tasks** across different agents/tools.
* It may call several agents **in sequence or parallel** to solve a complex query.

👉 Think of it like a **conductor of an orchestra**: makes sure violin, drums, and piano play together correctly.

<br>

---

<br>

## 3. **Workflow Difference**

### **Triage Agent**

```
User → Triage Agent → Decides → Specialist Agent (HR, Finance, IT, etc.)
```

### **Orchestrator Agent**

```
User → Orchestrator Agent → Calls Agent A → Calls Agent B → Combines results → User
```

<br>

---

<br>

## 4. **Best Use Cases**

### ✅ Triage Agent

* **Simple routing problems** (pick the right expert).
* Example use cases:

  * Customer support → “Is this IT, HR, or Finance?”
  * Medical chatbot → send to correct specialist bot.
  * Company helpdesk → route queries to the right department.

---

### ✅ Orchestrator Agent

* **Complex multi-step workflows** requiring coordination.
* Example use cases:

  * Travel booking assistant:

    * Ask flight agent → ask hotel agent → ask car rental agent → combine → give full itinerary.
  * Research assistant:

    * Collect data → analyze → summarize → generate report.
  * Business automation:

    * Fetch sales → calculate growth → email report.

<br>

---

<br>

## 5. **Analogy**

* **Triage Agent** = Receptionist in a hospital → “You go to Doctor A, you go to Doctor B.”
* **Orchestrator Agent** = Project Manager → “Doctor A checks blood test, Doctor B checks x-ray, then I’ll combine results.”

<br>

---

<br>

## 6. **Cheat-Sheet (Comparison Table)**

| Feature         | Triage Agent                     | Orchestrator Agent                |
| --------------- | ---------------------------------- | ------------------------------------- |
| Role            | Classifier / Router                | Planner / Coordinator                 |
| Decision-making | Chooses the **right single agent** | Manages **multiple agents/tools**     |
| Complexity      | Simple (one step)                  | Complex (multi-step workflows)        |
| Use Case        | Routing queries                    | Coordinating tasks                    |
| Analogy         | Hospital receptionist              | Orchestra conductor / project manager |

---

✅ **Summary**:

* A **Triage Agent** = a **router** that directs queries to the right expert agent.
* An **Orchestrator Agent** = a **coordinator** that manages multiple agents/tools to complete a workflow.
